In [ ]:
    L, K = calculate_periodic_points(f,g)
    if len(L) != len(K):  # checks maps have the same number of fixed points
        return []
    mult_to_point_L = {}
    mult_to_point_K = {}
    point_to_mult_L = {}
    point_to_mult_K = {}
    level = 0
    initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    repeated_mult_L = {}
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            pass
        elif len(mult_to_point_K[mult_L]) != repeated:
            pass
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]
    #print('repeated_mult:', repeated_mult_L)
    d = len(L)
    r = f.domain().base_ring()
    more = True
    source = []
    corresponding = []
    more = find_independent_set(repeated_mult_L, source, corresponding)
    #print('corresponding:', corresponding)
    i_repeated_mult = deepcopy(repeated_mult_L)
    a_repeated_mult = {}
    found_no_more = True
    while more:
        level += 1
        more, found_no_more, source, corresponding = (i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult)
        if found_no_more:
            raise ValueError('no more rational preimages')
        i_repeated_mult = deepcopy(a_repeated_mult)
        a_repeated_mult = {}
        found_no_more = True
        #print('i_repeated_mult:', i_repeated_mult)
    #print('source:', source)
    #print('mult_to_point_L:', mult_to_point_L)
    #print('mult_to_point_K:', mult_to_point_K)
    #print('point_to_mult_L:', point_to_mult_L)
    #print('point_to_mult_K:', point_to_mult_K)
    possible_targets = []
    arrangements_iterators = []
    initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)
    Conj = []
    find_conjugations(arrangements_iterators, possible_targets, source, Conj)
    return Conj

In [49]:
from itertools import product
from copy import deepcopy

In [1]:
def isLinearlyIndependent(S):
    M = matrix([list(t) for t in S])
    if len(S) < n+1:
        if M.rank() == len(S):
            return True
        return False
    else:
        if any(l == 0 for l in M.minors(n+1)):
            return False
        return True

In [2]:
def calculate_periodic_points(f, g):
    return Set(f.periodic_points(1)), Set(g.periodic_points(1))

In [18]:
def initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L):
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, 0)
        point_to_mult_K[K[i]] = (mult_K, 0)
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            pass
        elif len(mult_to_point_K[mult_L]) != repeated:
            pass
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]

In [60]:
def find_independent_set(repeated_mult_L, source, corresponding):
    more = True
    for r in repeated_mult_L:
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    return more

In [ ]:
def find_preimages(i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult):
    for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    more = find_independent_set(repeated_mult_L, source, corresponding)
                if not more:
                    break
            if not more:
                break
    return more, found_no_more, source, corresponding

In [5]:
def initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators):
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    for lst in possible_targets:
        arrangements_iterators.append(Arrangements(range(len(lst[0])), lst[1]))

In [66]:
def find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g):
    for tup in product(*arrangements_iterators):
        current_target = []
        for i in range(len(tup)):
            for j in tup[i]:
                current_target.append(possible_targets[i][0][j])
        #print('current_target:', current_target)
        try:
            phi = f.domain().point_transformation_matrix(current_target, source)
            #print('phi:', phi)
            #print('phi inverse', phi.inverse())
            #print([phi*pnt for pnt in current_target])
            #%time f.conjugate(phi) == g
            if f.conjugate(phi) == g:
                Conj.append(phi)
        except ValueError:
            pass

In [9]:
P.<x,y,z> = ProjectiveSpace(QQ, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
g = f

In [12]:
L, K = calculate_periodic_points(f,g)

In [58]:
%%timeit
mult_to_point_L = {}
mult_to_point_K = {}
point_to_mult_L = {}
point_to_mult_K = {}
repeated_mult_L = {}
level = 0
initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L)

45.3 ms ± 769 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [61]:
%%timeit
source = []
corresponding = []
n = f.domain().dimension_relative()
more = find_independent_set(repeated_mult_L, source, corresponding)

390 µs ± 6.56 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [40]:
i_repeated_mult = deepcopy(repeated_mult_L)
a_repeated_mult = {}
found_no_more = True
while more:
    level += 1
    more, found_no_more, source, corresponding = (i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult)
    if found_no_more:
        raise ValueError('no more rational preimages')
    i_repeated_mult = deepcopy(a_repeated_mult)
    a_repeated_mult = {}
    found_no_more = True

In [62]:
%%timeit
possible_targets = []
arrangements_iterators = []
initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)

3.86 µs ± 38.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [67]:
%%timeit
Conj = []
find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g)

CPU times: user 511 µs, sys: 0 ns, total: 511 µs
Wall time: 514 µs
CPU times: user 557 µs, sys: 0 ns, total: 557 µs
Wall time: 560 µs
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.83 ms
CPU times: user 2.2 ms, sys: 0 ns, total: 2.2 ms
Wall time: 2.21 ms
CPU times: user 759 µs, sys: 0 ns, total: 759 µs
Wall time: 762 µs
CPU times: user 1.48 ms, sys: 0 ns, total: 1.48 ms
Wall time: 1.49 ms
CPU times: user 2.03 ms, sys: 0 ns, total: 2.03 ms
Wall time: 2.04 ms
CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 2 ms
CPU times: user 802 µs, sys: 0 ns, total: 802 µs
Wall time: 806 µs
CPU times: user 1.33 ms, sys: 0 ns, total: 1.33 ms
Wall time: 1.34 ms
CPU times: user 1.44 ms, sys: 0 ns, total: 1.44 ms
Wall time: 1.44 ms
CPU times: user 1.54 ms, sys: 0 ns, total: 1.54 ms
Wall time: 1.54 ms
CPU times: user 1.56 ms, sys: 0 ns, total: 1.56 ms
Wall time: 1.57 ms
CPU times: user 2.24 ms, sys: 0 ns, total: 2.24 ms
Wall time: 2.25 ms
CPU times: user 1.95 ms, sys: 0 ns, t

CPU times: user 4.41 ms, sys: 0 ns, total: 4.41 ms
Wall time: 3.5 ms
CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 1.65 ms
CPU times: user 1.96 ms, sys: 0 ns, total: 1.96 ms
Wall time: 1.96 ms
CPU times: user 2.81 ms, sys: 0 ns, total: 2.81 ms
Wall time: 2.61 ms
CPU times: user 2.27 ms, sys: 0 ns, total: 2.27 ms
Wall time: 2.13 ms
CPU times: user 3.08 ms, sys: 0 ns, total: 3.08 ms
Wall time: 2.92 ms
CPU times: user 2.11 ms, sys: 0 ns, total: 2.11 ms
Wall time: 2.1 ms
CPU times: user 786 µs, sys: 0 ns, total: 786 µs
Wall time: 789 µs
CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 1.79 ms
CPU times: user 694 µs, sys: 0 ns, total: 694 µs
Wall time: 698 µs
CPU times: user 2.23 ms, sys: 0 ns, total: 2.23 ms
Wall time: 2.24 ms
CPU times: user 1.97 ms, sys: 0 ns, total: 1.97 ms
Wall time: 1.98 ms
CPU times: user 2.19 ms, sys: 0 ns, total: 2.19 ms
Wall time: 2.19 ms
CPU times: user 762 µs, sys: 0 ns, total: 762 µs
Wall time: 765 µs
CPU times: user 2.22 ms, sys: 0

CPU times: user 717 µs, sys: 0 ns, total: 717 µs
Wall time: 720 µs
CPU times: user 606 µs, sys: 0 ns, total: 606 µs
Wall time: 609 µs
CPU times: user 1.9 ms, sys: 0 ns, total: 1.9 ms
Wall time: 1.9 ms
CPU times: user 1.6 ms, sys: 0 ns, total: 1.6 ms
Wall time: 1.6 ms
CPU times: user 531 µs, sys: 0 ns, total: 531 µs
Wall time: 533 µs
CPU times: user 651 µs, sys: 0 ns, total: 651 µs
Wall time: 653 µs
CPU times: user 1.62 ms, sys: 0 ns, total: 1.62 ms
Wall time: 1.62 ms
CPU times: user 1.88 ms, sys: 0 ns, total: 1.88 ms
Wall time: 1.89 ms
CPU times: user 1.64 ms, sys: 0 ns, total: 1.64 ms
Wall time: 1.65 ms
CPU times: user 504 µs, sys: 0 ns, total: 504 µs
Wall time: 506 µs
CPU times: user 1.55 ms, sys: 0 ns, total: 1.55 ms
Wall time: 1.55 ms
CPU times: user 355 µs, sys: 0 ns, total: 355 µs
Wall time: 358 µs
CPU times: user 2.36 ms, sys: 0 ns, total: 2.36 ms
Wall time: 2.36 ms
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.84 ms
CPU times: user 1.9 ms, sys: 0 ns, total: 1.

Wall time: 1.92 ms
CPU times: user 2.07 ms, sys: 0 ns, total: 2.07 ms
Wall time: 2.07 ms
CPU times: user 1.96 ms, sys: 0 ns, total: 1.96 ms
Wall time: 1.96 ms
CPU times: user 1.87 ms, sys: 0 ns, total: 1.87 ms
Wall time: 1.87 ms
CPU times: user 765 µs, sys: 0 ns, total: 765 µs
Wall time: 768 µs
CPU times: user 728 µs, sys: 0 ns, total: 728 µs
Wall time: 731 µs
CPU times: user 1.65 ms, sys: 0 ns, total: 1.65 ms
Wall time: 1.65 ms
CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2 ms
CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2.01 ms
CPU times: user 3.25 ms, sys: 0 ns, total: 3.25 ms
Wall time: 3.16 ms
CPU times: user 1.98 ms, sys: 0 ns, total: 1.98 ms
Wall time: 1.99 ms
CPU times: user 2.56 ms, sys: 0 ns, total: 2.56 ms
Wall time: 2.48 ms
CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 3.9 ms
CPU times: user 559 µs, sys: 0 ns, total: 559 µs
Wall time: 563 µs
CPU times: user 608 µs, sys: 0 ns, total: 608 µs
Wall time: 611 µs
CPU times: user 1.44 ms, sys:

Wall time: 1.92 ms
CPU times: user 1.8 ms, sys: 0 ns, total: 1.8 ms
Wall time: 1.81 ms
CPU times: user 1.68 ms, sys: 0 ns, total: 1.68 ms
Wall time: 1.69 ms
CPU times: user 1.69 ms, sys: 0 ns, total: 1.69 ms
Wall time: 1.69 ms
CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 1.79 ms
CPU times: user 1.82 ms, sys: 0 ns, total: 1.82 ms
Wall time: 1.82 ms
CPU times: user 742 µs, sys: 0 ns, total: 742 µs
Wall time: 746 µs
CPU times: user 533 µs, sys: 0 ns, total: 533 µs
Wall time: 536 µs
CPU times: user 449 µs, sys: 0 ns, total: 449 µs
Wall time: 452 µs
CPU times: user 1.57 ms, sys: 0 ns, total: 1.57 ms
Wall time: 1.58 ms
CPU times: user 1.53 ms, sys: 0 ns, total: 1.53 ms
Wall time: 1.54 ms
CPU times: user 1.84 ms, sys: 0 ns, total: 1.84 ms
Wall time: 1.84 ms
CPU times: user 1.82 ms, sys: 0 ns, total: 1.82 ms
Wall time: 1.83 ms
CPU times: user 1.93 ms, sys: 0 ns, total: 1.93 ms
Wall time: 1.94 ms
CPU times: user 2.12 ms, sys: 0 ns, total: 2.12 ms
Wall time: 2.13 ms
CPU times: u

...

Wall time: 1.92 ms
CPU times: user 1.64 ms, sys: 0 ns, total: 1.64 ms
Wall time: 1.64 ms
CPU times: user 1.32 ms, sys: 0 ns, total: 1.32 ms
Wall time: 1.33 ms
CPU times: user 1.38 ms, sys: 0 ns, total: 1.38 ms
Wall time: 1.38 ms
CPU times: user 708 µs, sys: 0 ns, total: 708 µs
Wall time: 710 µs
CPU times: user 712 µs, sys: 0 ns, total: 712 µs
Wall time: 715 µs
CPU times: user 1.56 ms, sys: 0 ns, total: 1.56 ms
Wall time: 1.57 ms
CPU times: user 1.05 ms, sys: 0 ns, total: 1.05 ms
Wall time: 1.06 ms
CPU times: user 2.25 ms, sys: 0 ns, total: 2.25 ms
Wall time: 2.25 ms
CPU times: user 1.19 ms, sys: 0 ns, total: 1.19 ms
Wall time: 1.2 ms
CPU times: user 651 µs, sys: 0 ns, total: 651 µs
Wall time: 654 µs
CPU times: user 2.09 ms, sys: 0 ns, total: 2.09 ms
Wall time: 2.09 ms
CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 1.79 ms
CPU times: user 357 µs, sys: 0 ns, total: 357 µs
Wall time: 359 µs
CPU times: user 503 µs, sys: 0 ns, total: 503 µs
Wall time: 506 µs
CPU times: user 1

CPU times: user 1.74 ms, sys: 0 ns, total: 1.74 ms
Wall time: 1.74 ms
CPU times: user 1.37 ms, sys: 0 ns, total: 1.37 ms
Wall time: 1.37 ms
CPU times: user 1.57 ms, sys: 0 ns, total: 1.57 ms
Wall time: 1.57 ms
CPU times: user 726 µs, sys: 0 ns, total: 726 µs
Wall time: 729 µs
CPU times: user 1.81 ms, sys: 0 ns, total: 1.81 ms
Wall time: 1.81 ms
CPU times: user 1.75 ms, sys: 0 ns, total: 1.75 ms
Wall time: 1.76 ms
CPU times: user 2.08 ms, sys: 0 ns, total: 2.08 ms
Wall time: 2.08 ms
CPU times: user 1.03 ms, sys: 0 ns, total: 1.03 ms
Wall time: 1.04 ms
CPU times: user 1.82 ms, sys: 0 ns, total: 1.82 ms
Wall time: 1.82 ms
CPU times: user 1.87 ms, sys: 0 ns, total: 1.87 ms
Wall time: 1.88 ms
CPU times: user 1.97 ms, sys: 0 ns, total: 1.97 ms
Wall time: 1.97 ms
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.83 ms
CPU times: user 1.56 ms, sys: 0 ns, total: 1.56 ms
Wall time: 1.56 ms
CPU times: user 1.76 ms, sys: 0 ns, total: 1.76 ms
Wall time: 1.76 ms
CPU times: user 1.84 ms

Wall time: 2.49 ms
CPU times: user 1.89 ms, sys: 0 ns, total: 1.89 ms
Wall time: 1.89 ms
CPU times: user 1.79 ms, sys: 0 ns, total: 1.79 ms
Wall time: 1.79 ms
CPU times: user 769 µs, sys: 0 ns, total: 769 µs
Wall time: 774 µs
CPU times: user 944 µs, sys: 0 ns, total: 944 µs
Wall time: 950 µs
CPU times: user 1.91 ms, sys: 0 ns, total: 1.91 ms
Wall time: 1.92 ms
CPU times: user 2 ms, sys: 0 ns, total: 2 ms
Wall time: 2.01 ms
CPU times: user 382 µs, sys: 0 ns, total: 382 µs
Wall time: 385 µs
CPU times: user 1.4 ms, sys: 0 ns, total: 1.4 ms
Wall time: 1.42 ms
CPU times: user 2.15 ms, sys: 0 ns, total: 2.15 ms
Wall time: 2.16 ms
CPU times: user 2.16 ms, sys: 0 ns, total: 2.16 ms
Wall time: 2.17 ms
CPU times: user 630 µs, sys: 0 ns, total: 630 µs
Wall time: 632 µs
CPU times: user 1.49 ms, sys: 0 ns, total: 1.49 ms
Wall time: 1.49 ms
CPU times: user 2.24 ms, sys: 0 ns, total: 2.24 ms
Wall time: 2.25 ms
CPU times: user 2.3 ms, sys: 0 ns, total: 2.3 ms
Wall time: 2.31 ms
CPU times: user 1.41 ms

CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.84 ms
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.83 ms
CPU times: user 1.29 ms, sys: 0 ns, total: 1.29 ms
Wall time: 1.29 ms
CPU times: user 1.31 ms, sys: 0 ns, total: 1.31 ms
Wall time: 1.31 ms
CPU times: user 439 µs, sys: 0 ns, total: 439 µs
Wall time: 442 µs
CPU times: user 0 ns, sys: 1.68 ms, total: 1.68 ms
Wall time: 1.68 ms
CPU times: user 1.7 ms, sys: 0 ns, total: 1.7 ms
Wall time: 1.71 ms
CPU times: user 1.8 ms, sys: 0 ns, total: 1.8 ms
Wall time: 1.8 ms
CPU times: user 1.85 ms, sys: 0 ns, total: 1.85 ms
Wall time: 1.86 ms
CPU times: user 1.73 ms, sys: 0 ns, total: 1.73 ms
Wall time: 1.73 ms
CPU times: user 1.63 ms, sys: 0 ns, total: 1.63 ms
Wall time: 1.64 ms
CPU times: user 1.71 ms, sys: 0 ns, total: 1.71 ms
Wall time: 1.72 ms
CPU times: user 1.36 ms, sys: 0 ns, total: 1.36 ms
Wall time: 1.36 ms
CPU times: user 1.58 ms, sys: 0 ns, total: 1.58 ms
Wall time: 1.59 ms
CPU times: user 1.02 ms, sys

Wall time: 1.41 ms
CPU times: user 695 µs, sys: 0 ns, total: 695 µs
Wall time: 697 µs
CPU times: user 764 µs, sys: 0 ns, total: 764 µs
Wall time: 769 µs
CPU times: user 1.63 ms, sys: 0 ns, total: 1.63 ms
Wall time: 1.63 ms
CPU times: user 657 µs, sys: 0 ns, total: 657 µs
Wall time: 660 µs
CPU times: user 1.99 ms, sys: 0 ns, total: 1.99 ms
Wall time: 1.99 ms
CPU times: user 1.06 ms, sys: 0 ns, total: 1.06 ms
Wall time: 1.06 ms
CPU times: user 610 µs, sys: 0 ns, total: 610 µs
Wall time: 613 µs
CPU times: user 2.09 ms, sys: 0 ns, total: 2.09 ms
Wall time: 2.09 ms
CPU times: user 3.04 ms, sys: 0 ns, total: 3.04 ms
Wall time: 2.79 ms
CPU times: user 383 µs, sys: 0 ns, total: 383 µs
Wall time: 386 µs
CPU times: user 798 µs, sys: 0 ns, total: 798 µs
Wall time: 803 µs
CPU times: user 1.83 ms, sys: 0 ns, total: 1.83 ms
Wall time: 1.83 ms
CPU times: user 1.38 ms, sys: 0 ns, total: 1.38 ms
Wall time: 1.39 ms
CPU times: user 357 µs, sys: 0 ns, total: 357 µs
Wall time: 359 µs
CPU times: user 1.53 m

In [71]:
t = list(product(*arrangements_iterators))

In [72]:
def test_conj(tup, possible_targets, source, f, g):
    current_target = []
    for i in range(len(tup)):
        for j in tup[i]:
            current_target.append(possible_targets[i][0][j])
    try:
        phi = f.domain().point_transformation_matrix(current_target, source)
        if f.conjugate(phi) == g:
            return phi
    except ValueError:
        pass

In [ ]:
parallel_data = []
for tup in t:
    parallel_data.append([tup, possible_targets, source, f, g])